In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-04,France,131903.0,0.0,25082.0,0.0,0.0,0.0
0,2020-05-04,China,84051.0,0.0,4664.0,0.0,0.0,0.0
0,2020-05-04,Italy,212580.0,0.0,29347.0,0.0,0.0,0.0
0,2020-05-04,Spain,218517.0,0.0,25504.0,0.0,0.0,0.0
0,2020-05-04,United States,1188404.0,0.0,68930.0,0.0,0.0,0.0
0,2020-05-04,World,3552698.0,0.0,251713.0,0.0,0.0,0.0
0,2020-05-04,United Kingdom,191279.0,0.0,28924.0,0.0,0.0,0.0
0,2020-05-04,Germany,163670.0,0.0,6747.0,0.0,0.0,0.0
0,2020-05-04,Iran,98223.0,0.0,6274.0,0.0,0.0,0.0
0,2020-05-04,Turkey,127710.0,0.0,3463.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-05,France,131903.0,131863.0,25082.0,25201.0,40.0,-119.0
0,2020-05-05,China,84051.0,83966.0,4664.0,4637.0,85.0,27.0
0,2020-05-05,Italy,212580.0,211938.0,29347.0,29079.0,642.0,268.0
0,2020-05-05,Spain,218517.0,218011.0,25504.0,25428.0,506.0,76.0
0,2020-05-05,United States,1188404.0,1180634.0,68930.0,68934.0,7770.0,-4.0
0,2020-05-05,World,3552698.0,3544222.0,251713.0,250977.0,8476.0,736.0
0,2020-05-05,United Kingdom,191279.0,190584.0,28924.0,28734.0,695.0,190.0
0,2020-05-05,Germany,163670.0,163860.0,6747.0,6831.0,-190.0,-84.0
0,2020-05-05,Iran,98223.0,98647.0,6274.0,6277.0,-424.0,-3.0
0,2020-05-05,Turkey,127710.0,127659.0,3463.0,3461.0,51.0,2.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")